In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=39f16b628d30dd7f96dc98ad66b5759d9852da1597d022853dacdd9218159d5e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import wget
import os
filename = []
output_directory = "/content/drive/MyDrive"
urls = ['https://raw.githubusercontent.com/mbzuai-nlp/M4/main/data/arxiv_bloomz.jsonl',
        'https://raw.githubusercontent.com/mbzuai-nlp/M4/main/data/arxiv_chatGPT.jsonl',
        'https://raw.githubusercontent.com/mbzuai-nlp/M4/main/data/arxiv_cohere.jsonl',
        'https://raw.githubusercontent.com/mbzuai-nlp/M4/main/data/arxiv_davinci.jsonl',
        'https://raw.githubusercontent.com/mbzuai-nlp/M4/main/data/arxiv_flant5.jsonl',
        'https://huggingface.co/datasets/NicolaiSivesind/ChatGPT-Research-Abstracts/raw/main/research_abstracts-final.csv']

for url in urls:
    output_filename = output_directory + '/' + os.path.basename(url) # get the full path of the file
    if os.path.exists(output_filename):
        os.remove(output_filename) # if exist, remove it directly
    filename.append(wget.download(url, out=output_filename))
print(filename)

['/content/drive/MyDrive/arxiv_bloomz.jsonl', '/content/drive/MyDrive/arxiv_chatGPT.jsonl', '/content/drive/MyDrive/arxiv_cohere.jsonl', '/content/drive/MyDrive/arxiv_davinci.jsonl', '/content/drive/MyDrive/arxiv_flant5.jsonl', '/content/drive/MyDrive/research_abstracts-final.csv']


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("google/t5-efficient-large-nl32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained("SamSJackson/paraphrase-dipper-no-ctx")
model = model.to(device)

config.json:   0%|          | 0.00/783 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# df = pd.read_json('/content/drive/MyDrive/arxiv_bloomz.jsonl', lines=True)

# paraphrased_all = []
# lexical = 20
# order = 40
# count = 0

# for df in df.itertuples():
#   prompt = f"lexical = {lexical}, order = {order} {df.machine_abstract}"
#   input_ids = tokenizer(
#     prompt,
#     return_tensors='pt',
#     padding="longest",
#     max_length=1000,
#     truncation=True,
#   ).to(device)

#   outputs = model.generate(
#       **input_ids,
#       top_p=0.75,
#       do_sample=True,
#       max_new_tokens=300,
#   )

#   response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#   response = f"{' '.join(response)}"

#   #print(response)
#   paraphrased = dict(source_id=df.source_id,title=df.title, abstract=df.abstract, paraphrased_abstract=response, model="bigscience bloomz", source="arxiv")
#   #print(str(paraphrased))
#   paraphrased_all.append(paraphrased)
#   count += 1
#   print(count, end=", ")



In [ ]:
# print(paraphrased_all)
# with open("/content/drive/MyDrive/arxiv_bloomz_paraphrased.jsonl", "w") as json_file:
#     pass
# with open("/content/drive/MyDrive/arxiv_bloomz_paraphrased.jsonl", "r+", encoding="utf-8") as jsonl_file:
#     for item in paraphrased_all:
#         json_string = json.dumps(item, ensure_ascii=False)
#         jsonl_file.write(json_string + "\n")

# Bloomz - Paraphrase Bloomz generated abstract

In [ ]:
import json
import pandas as pd

# Read JSONL file into a DataFrame
df = pd.read_json('/content/drive/MyDrive/arxiv_bloomz.jsonl', lines=True)

paraphrased_all = []
lexical = 20
order = 40
count = 0

# Iterate over each row in DataFrame
for entry in df.itertuples():
    prompt = f"lexical = {lexical}, order = {order} {entry.machine_abstract}"
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding="longest",
        max_length=1000,
        truncation=True,
    ).to(device)

    outputs = model.generate(
        **input_ids,
        top_p=0.75,
        do_sample=True,
        max_new_tokens=300,
    )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    response = ' '.join(response)

    paraphrased = dict(
        source_id=entry.source_id,
        title=entry.title,
        abstract=entry.abstract,
        machine_abstract=entry.machine_abstract,
        paraphrased_abstract=response,
        model="bigscience bloomz",
        source="arxiv"
    )

    paraphrased_all.append(paraphrased)
    count += 1
    print(count, end=", ")

    # Save every 50 lines
    if count % 50 == 0:
        with open("/content/drive/MyDrive/arxiv_bloomz_paraphrased_full.jsonl", "a+", encoding="utf-8") as jsonl_file:
            for item in paraphrased_all:
                json_string = json.dumps(item, ensure_ascii=False)
                jsonl_file.write(json_string + "\n")
        paraphrased_all = []  # Clear the list after saving

# Save any remaining entries after the loop completes
if paraphrased_all:
    with open("/content/drive/MyDrive/arxiv_bloomz_paraphrased_full.jsonl", "a+", encoding="utf-8") as jsonl_file:
        for item in paraphrased_all:
            json_string = json.dumps(item, ensure_ascii=False)
            jsonl_file.write(json_string + "\n")


1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222

In [ ]:
# df = pd.read_json('/content/drive/MyDrive/arxiv_bloomz.jsonl', lines=True)

# paraphrased_all = []
# lexical = 20
# order = 40
# count = 0

# for df in df.itertuples():
#   prompt = f"lexical = {lexical}, order = {order} {df.machine_abstract}"
#   input_ids = tokenizer(
#     prompt,
#     return_tensors='pt',
#     padding="longest",
#     max_length=1000,
#     truncation=True,
#   ).to(device)

#   outputs = model.generate(
#       **input_ids,
#       top_p=0.75,
#       do_sample=True,
#       max_new_tokens=300,
#   )

#   response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#   response = f"{' '.join(response)}"

#   #print(response)
#   paraphrased = dict(source_id=df.source_id,title=df.title, abstract=df.abstract, machine_abstract = df.machine_abstract, paraphrased_abstract=response, model="bigscience bloomz", source="arxiv")
#   #print(str(paraphrased))
#   paraphrased_all.append(paraphrased)
#   count += 1
#   print(count, end=", ")



1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 

KeyboardInterrupt: 

In [ ]:
# print(paraphrased_all)
# with open("/content/drive/MyDrive/arxiv_bloomz_paraphrased_full.jsonl", "w") as json_file:
#     pass
# with open("/content/drive/MyDrive/arxiv_bloomz_paraphrased_full.jsonl", "r+", encoding="utf-8") as jsonl_file:
#     for item in paraphrased_all:
#         json_string = json.dumps(item, ensure_ascii=False)
#         jsonl_file.write(json_string + "\n")

[{'source_id': 704.0007, 'title': 'Polymer Quantum Mechanics and its Continuum Limit', 'abstract': '  A rather non-standard quantum representation of the canonical commutation\nrelations of quantum mechanics systems, known as the polymer representation has\ngained some attention in recent years, due to its possible relation with Planck\nscale physics. In particular, this approach has been followed in a symmetric\nsector of loop quantum gravity known as loop quantum cosmology. Here we explore\ndifferent aspects of the relation between the ordinary Schroedinger theory and\nthe polymer description. The paper has two parts. In the first one, we derive\nthe polymer quantum mechanics starting from the ordinary Schroedinger theory\nand show that the polymer description arises as an appropriate limit. In the\nsecond part we consider the continuum limit of this theory, namely, the reverse\nprocess in which one starts from the discrete theory and tries to recover back\nthe ordinary Schroedinger 

# Bloomz - Paraphrase human generated abstract

In [ ]:
df = pd.read_json('/content/drive/MyDrive/arxiv_bloomz.jsonl', lines=True)

paraphrased_all = []
lexical = 20
order = 40
count = 0

for df in df.itertuples():
  prompt = f"lexical = {lexical}, order = {order} {df.abstract}"
  input_ids = tokenizer(
    prompt,
    return_tensors='pt',
    padding="longest",
    max_length=1000,
    truncation=True,
  ).to(device)

  outputs = model.generate(
      **input_ids,
      top_p=0.75,
      do_sample=True,
      max_new_tokens=300,
  )

  response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  response = f"{' '.join(response)}"

  #print(response)
  paraphrased = dict(source_id=df.source_id,title=df.title, abstract=df.abstract, machine_abstract = df.machine_abstract, paraphrased_abstract=response, model="bigscience bloomz", source="arxiv")
  #print(str(paraphrased))
  paraphrased_all.append(paraphrased)
  count += 1
  print(count, end=", ")



1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 

KeyboardInterrupt: 

In [ ]:
print(paraphrased_all)
with open("/content/drive/MyDrive/arxiv_bloomz_paraphrased_ht.jsonl", "w") as json_file:
    pass
with open("/content/drive/MyDrive/arxiv_bloomz_paraphrased_ht.jsonl", "r+", encoding="utf-8") as jsonl_file:
    for item in paraphrased_all:
        json_string = json.dumps(item, ensure_ascii=False)
        jsonl_file.write(json_string + "\n")

[{'source_id': 704.0007, 'title': 'Polymer Quantum Mechanics and its Continuum Limit', 'abstract': '  A rather non-standard quantum representation of the canonical commutation\nrelations of quantum mechanics systems, known as the polymer representation has\ngained some attention in recent years, due to its possible relation with Planck\nscale physics. In particular, this approach has been followed in a symmetric\nsector of loop quantum gravity known as loop quantum cosmology. Here we explore\ndifferent aspects of the relation between the ordinary Schroedinger theory and\nthe polymer description. The paper has two parts. In the first one, we derive\nthe polymer quantum mechanics starting from the ordinary Schroedinger theory\nand show that the polymer description arises as an appropriate limit. In the\nsecond part we consider the continuum limit of this theory, namely, the reverse\nprocess in which one starts from the discrete theory and tries to recover back\nthe ordinary Schroedinger 

# GPT - Paraphrase GPT generated abstract

In [ ]:
df = pd.read_json('/content/drive/MyDrive/arxiv_chatGPT.jsonl', lines=True)

paraphrased_all = []
lexical = 20
order = 40
count = 0

for df in df.itertuples():
  prompt = f"lexical = {lexical}, order = {order} {df.machine_text}"
  input_ids = tokenizer(
    prompt,
    return_tensors='pt',
    padding="longest",
    max_length=1000,
    truncation=True,
  ).to(device)

  outputs = model.generate(
      **input_ids,
      top_p=0.75,
      do_sample=True,
      max_new_tokens=300,
  )

  response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  response = f"{' '.join(response)}"

  #print(response)
  paraphrased = dict(source_ID=df.source_ID,prompt=df.prompt, human_text=df.human_text, machine_text = df.machine_text, paraphrased_abstract=response, model="gpt-3.5-turbo", source="arxiv")
  #print(str(paraphrased))
  paraphrased_all.append(paraphrased)
  count += 1
  print(count, end=", ")



1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 

KeyboardInterrupt: 

In [ ]:
print(paraphrased_all)
with open("/content/drive/MyDrive/arxiv_chatGPT_paraphrased_mt.jsonl", "w") as json_file:
    pass
with open("/content/drive/MyDrive/arxiv_chatGPT_paraphrased_mt.jsonl", "r+", encoding="utf-8") as jsonl_file:
    for item in paraphrased_all:
        json_string = json.dumps(item, ensure_ascii=False)
        jsonl_file.write(json_string + "\n")

[{'source_ID': 704.0007, 'prompt': 'Generate a 150-220-word abstract for work with title: Polymer Quantum Mechanics and its Continuum Limit. Describe the results obtained,  problem work attempts to solve, and the key ideas and methodology in a formal academic and scientific writing voice. Use the first plural person form. Use active voice.', 'human_text': '  A rather non-standard quantum representation of the canonical commutation\nrelations of quantum mechanics systems, known as the polymer representation has\ngained some attention in recent years, due to its possible relation with Planck\nscale physics. In particular, this approach has been followed in a symmetric\nsector of loop quantum gravity known as loop quantum cosmology. Here we explore\ndifferent aspects of the relation between the ordinary Schroedinger theory and\nthe polymer description. The paper has two parts. In the first one, we derive\nthe polymer quantum mechanics starting from the ordinary Schroedinger theory\nand sh

In [ ]:
import json
import pandas as pd

# Read JSONL file into a DataFrame
df = pd.read_json('/content/drive/MyDrive/arxiv_chatGPT.jsonl', lines=True)

paraphrased_all = []
lexical = 20
order = 40
count = 0

# Iterate over each row in DataFrame
#

# Start from the 51st line
# for entry in df.iloc[50:].itertuples():
for entry in df.itertuples():
    prompt = f"lexical = {lexical}, order = {order} {entry.machine_text}"
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding="longest",
        max_length=1000,
        truncation=True,
    ).to(device)

    outputs = model.generate(
        **input_ids,
        top_p=0.75,
        do_sample=True,
        max_new_tokens=300,
    )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    response = ' '.join(response)

    paraphrased = dict(
        source_ID=entry.source_ID,
		    prompt=entry.prompt,
		    human_text=entry.human_text,
        machine_text=entry.machine_text,
        paraphrased_abstract=response,
        model="gpt-3.5-turbo",
        source="arxiv"
    )

    paraphrased_all.append(paraphrased)
    count += 1
    print(count, end=", ")

    # Save every 50 lines
    if count % 50 == 0:
        with open("/content/drive/MyDrive/arxiv_chatGPT_paraphrased_mt_2.jsonl", "a+", encoding="utf-8") as jsonl_file:
            for item in paraphrased_all:
                json_string = json.dumps(item, ensure_ascii=False)
                jsonl_file.write(json_string + "\n")
        paraphrased_all = []  # Clear the list after saving

# Save any remaining entries after the loop completes
if paraphrased_all:
    with open("/content/drive/MyDrive/arxiv_chatGPT_paraphrased_mt_2.jsonl", "a+", encoding="utf-8") as jsonl_file:
        for item in paraphrased_all:
            json_string = json.dumps(item, ensure_ascii=False)
            jsonl_file.write(json_string + "\n")


1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222

# Cohere - Paraphrase Cohere generated abstract

In [ ]:
import json
import pandas as pd

# Read JSONL file into a DataFrame
df = pd.read_json('/content/drive/MyDrive/arxiv_cohere.jsonl', lines=True)

paraphrased_all = []
lexical = 20
order = 40
count = 0

# Iterate over each row in DataFrame
#

# Start from the 51st line
# for entry in df.iloc[50:].itertuples():
for entry in df.itertuples():
    prompt = f"lexical = {lexical}, order = {order} {entry.machine_text}"
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        padding="longest",
        max_length=1000,
        truncation=True,
    ).to(device)

    outputs = model.generate(
        **input_ids,
        top_p=0.75,
        do_sample=True,
        max_new_tokens=300,
    )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    response = ' '.join(response)

    paraphrased = dict(
        source_id=entry.source_id,
		    prompt=entry.prompt,
		    human_text=entry.human_text,
        machine_text=entry.machine_text,
        paraphrased_abstract=response,
        model="command-xlarge-nightly",
        source="arxiv"
    )

    paraphrased_all.append(paraphrased)
    count += 1
    print(count, end=", ")

    # Save every 50 lines
    if count % 50 == 0:
        with open("/content/drive/MyDrive/arxiv_cohere_paraphrased_mt_2.jsonl", "a+", encoding="utf-8") as jsonl_file:
            for item in paraphrased_all:
                json_string = json.dumps(item, ensure_ascii=False)
                jsonl_file.write(json_string + "\n")
        paraphrased_all = []  # Clear the list after saving

# Save any remaining entries after the loop completes
if paraphrased_all:
    with open("/content/drive/MyDrive/arxiv_cohere_paraphrased_mt_2.jsonl", "a+", encoding="utf-8") as jsonl_file:
        for item in paraphrased_all:
            json_string = json.dumps(item, ensure_ascii=False)
            jsonl_file.write(json_string + "\n")


1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222